In [ ]:
run statistic.ipynb

In [ ]:
#
# Post-fit analysis package: 
# simple estimators of effective masses, ratio plots for 3pt correlators. 
#
import sys
import numpy as np
import re
import os
import pandas as pds
import math

# Post-fit analysis class
#   is2pt: True / False / string for 2pt data / non-2pt data / the identifier (string) of 2pt data
class ML_Analyze_PDF: 
    def __init__(self, params):
        self.keys = []# ['p', 'z', 't']
        self.scount = 0
        self.seriesID = None #'s'+str(self.scount)
        self.atables = [] #index = self.seriesID, columns = self.keys)
        self.afuncs = {}
        try:
            self.is2pt = str(params['spectrum'])
            if len(list(self.is2pt)) == 0:
                self.is2pt = str(None)
           #if (self.is2pt is not True) and (self.is2pt is not False):
            #    self.is2pt = str(params['spectrum'])
        except:
            self.is2pt = str(None)
        self.NA = np.NAN
        try:
            self.nsrc = params['nsrc']
        except:
            self.nsrc = 1
        try:
            self.ave = params['effmass']
            if self.ave is False:
                self.ave = params['ratio']
        except:
            self.ave = False
        if self.nsrc == 1:
            self.ave = False
        print('self.ave is {:}'.format(self.ave))
        try:
            self.ratio_mp = params['neg_p_2pt']
        except:
            self.ratio_mp = False
        try:
            self.checkBC = params['checkbc']
        except:
            self.checkBC = False
        self.pltcid = 0
        self.zid = 2
        self.pid = 1
        self.oid = 0
        self.table_open = False
        self.DEBUG = False
        return
    
    # free memory, reset class attributes to default
    def finalize(self):
        if self.table_open:
            self.merge_table()
        del self.atables
        self.__init__(None)
        return
    
    # Add in customized features of the data for each set of fit, stored in keys (and elem)
    # keys: names of data features, string, dictionary or list
    # elem: values of data features
    def extend_table(self, keys = None, elem = None):
        if self.scount == 0:
            return
        if elem is None:
            if keys is None:
                return
            if isinstance(keys, dict):
                for key in keys:
                    if key in self.keys:
                        print("ML_Analyze_PDF Warning: no overwriting to exsisting entry {:}".format(key))
                    else:
                        if keys[key] is not None:
                            self.wkrtable[-1].insert(key, keys[key])
                        self.keys.append(key)
            else:
                print("ML_Analyze_PDF Error: try to extend table with unidentified elements!")
                sys.exit(1)
            return
        if isinstance(keys, str):
            if elem is not None:
                self.wkrtable[-1].insert(len(self.wkrtable[-1].columns), keys, elem)
                if keys not in self.keys:
                    self.keys.append(keys)
        else:
            for key in keys:
                if key in self.keys:
                    print("ML_Analyze_PDF Warning: no overwrite to exsisting entry {:}".format(key))
                else:
                    if elem[key] is not None:
                        self.wkrtable[-1].insert(key, elem[key])
                    self.keys.append(key)
        return
    
    # add to the post-fit history a new set of fit identified with:
    #  NT: temporal lattice size
    #  ndata / ntrn / nbc: number of total data/ training data/ bias correction data 
    #  pztY: list of data Y's ids: momentum, z-link, time slices
    #  pztX: list of data X's ids: momentum, z-link, time slices
    def add_table(self, NT, ndata, ntrn, nbc, pztY, pztX=None):
        if self.table_open:
            self.merge_table()
        self.seriesID = 's'+str(self.scount)
        self.wkrtable = []
        pzY = []
        tY = []
        if isinstance(NT, int) is False:
            NTl = []
        else:
            NTl = NT
        for tg in pztY:
            #flag = True
            ytag = tg.split('+t+')
            pz = ytag[0]
            if pz in pzY:
                continue
            pzY.append(pz)
            if isinstance(NTl, list):
                NTl.append(NT[pz])
            t=[]
            for tg2 in pztY:
                if pz+'+t+' in tg2 and (int(tg2.split('+')[-2]) not in t):
                    t.append(int(tg2.split('+')[-2]))
            t.sort()
            tY.append(t)
            del t
        #self.atables.append(pds.DataFrame(data=[NT, ndata, ntrn, nbc+ntrn, list(pztX), tY], index = pzY, 
                #                                  columns=['NT', 'ndata', 'ntrn', 'nlbl', 'pztX', 'tY']))
        if pztX is None:
            pztXl = None
        elif isinstance(pztX, dict):
            pztXl = [list(pztX[pz]) for pz in pztX]
        else:
            pztXl = [pztX]
        self.tableframe = pds.DataFrame(data={'NT': NTl, 'ndata': ndata, 'ntrn': ntrn, 'nlbl': nbc+ntrn, 
                                              'pztX': pztXl, 'tY': [tY]}, index = pzY) 
                                             #     columns=['NT', 'ndata', 'ntrn', 'nlbl', 'pztX', 'tY'])
        self.keyf = ['NT', 'ndata', 'ntrn', 'nlbl', 'pztX', 'tY']
        self.keys = []
        if False:#if pztX is not None:
            self.keyf.append('pztX') 
        self.table_open = True
        self.datatmp = {}
        if self.ave:
            self.datatmpave = {}
        return
    
    def drop_table(self, indx = None):
        if indx is None:
            indx = -1
        ln = len(self.atables)
        if ((indx+ln)%ln == ln-1) and self.table_open:
            self.merge_table()
            del self.tableframe
        self.atables.pop(indx)
        return
    
    # Append to the current set of fit the fit results
    def append_table(self, ddir):
        self.wkrtable.append(pds.DataFrame(self.tableframe))
        self.keys = list(self.keyf)
        if len(self.datatmp) > 0:
            #for key in self.datatmp:
             #   del self.datatmp[key]
            #pf = open(ddir+'/'+self.seriesID+'.'+str(len(self.wkrtable)-1), 'a')
            #pf.write("{:}\n\n".format(self.datatmp))
            del self.datatmp
            self.datatmp = {}
            if self.ave:
                del self.datatmpave
                self.datatmpave = {}
        return
    
    def pop_table(self, indx = None):
        if indx is None:
            indx = -1
        try:
            self.wkrtable.pop(indx)
        except:
            print("Warning: Failed attempt to delete the worker table {:}".format(indx))
        return
    
    # Merge all fits from the current set of fit
    def merge_table(self):
        if self.table_open is False:
            print("Warning: No work table to merge!")
            return
        if len(self.wkrtable) > 1:
            self.atables.append(self.wkrtable[0].append(list(self.wkrtable[1:])))
        else:
            self.atables.append(self.wkrtable[0])
        self.scount += 1
        del self.wkrtable
        #self.wkrtable = []
        del self.keys
        del self.keyf
        if len(self.datatmp) > 0:
            #for key in self.datatmp:
             #   del self.datatmp[key]
            del self.datatmp
        self.table_open = False
        return
      
    def setup_ffunc(self, func, key = None):
        if isinstance(func, dict):
            for key in func:
                self.afuncs[key] = func[key]
        elif key is None:
            print("ML_Analyze_PDF Error: try to add functions with unidentified names!")
            sys.exit(1)
        else:
            if isinstance(key, (tuple, list)): 
                assert(len(func)==len(key))
                for k in key:
                    self.afuncs[k] = func[k]
            else:
                self.afuncs[key] = func
        return
    
    # Perform dias correction on the predicted Y's; tag: list of data id's
    def biascorrt_cov(self, tag, indx = None):
        if indx is None:
            indx = -1
        cnamel = ['ycov']
        if self.ave:
            cnamel.append('ycovave')
        for cname, ri in product(cnamel, ['R', 'I']):
            if cname+ri not in self.wkrtable[indx].columns:
                print("ML_Analyze_PDF Error: Missing data covariance matrix!")
                sys.exit(1)
        tab = self.wkrtable[indx]
        for ri in ['R', 'I']:
            if 'bcycov'+ri not in tab.columns:
                tab.insert(len(tab.columns), 'bcycov'+ri, None)
            if self.ave and ('bcycovave'+ri not in tab.columns):
                tab.insert(len(tab.columns), 'bcycovave'+ri, None)
        for tg in tag:
            pz = tg.split('+t+')[0]
            ri = tg.split('+')[-1]
            if tab.loc[pz, 'bcycov'+ri] is not None:
                continue
            for cname in cnamel:
                cov = tab.loc[pz,cname+ri]
                if cov is None: 
                    continue
                ny = int(cov.shape[0]/2)
                assert(cov.shape[1] == cov.shape[0] == 2*ny)
                tab['bc'+cname+ri][pz] = (2.*cov[:ny, :ny] + cov[ny:, ny:] - cov[:ny,ny:] - cov[ny:,:ny])
                       #  -np.matmul(cov[:ny,ny:], cov[ny:,:ny])-np.matmul(cov[ny:,:ny], cov[:ny,ny:]))
        if self.DEBUG:
            print(self.wkrtable[indx])
        return
    
    # Add to the data storage for post-fit analysis: 
    #   tag: data ids
    #   data: list of data (same length as tag)
    #   dscale: data normalization factors (scalar of vector of same length as tag)
    #   dtrn / dbc / dunlbl : list of traing / bias correction / unlabeled data , an alternative to 'data'
    #   pred: True / False for predicted / observed data
    #   NT: temporal lattice size
    #   is2pt: if the data are 2pt correlators
    #   overwrite: True / False for overwriting / no overwriting of existing data w. identical id
    #   indx: None for working on the most recent fit
    def add_data(self, tag, data=None, dscale = None, dtrn=None, dbc=None, dunlbl=None, pred=True, NT=None, pztX=None, is2pt = False, overwrite = False, indx = None):
        if is2pt and (self.is2pt is False):
            return
        if indx is None:
            indx = -1
        tab = self.wkrtable[indx]
        if pred is False:
            name = 'Yobs'
        else:
            name = 'Ypdt'
        if name not in self.datatmp:
            self.datatmp[name] = {}
            if self.ave:
                self.datatmpave[name] = {}
        elif overwrite:
            del self.datatmp[name] 
            self.datatmp[name] = {}
            if self.ave:
                del self.datatmpave[name]
                self.datatmpave[name] = {}
        else:
            #tag1 = list(tag)
            tag=list(tag)
            for i in range(len(tag)):
                if tag[i].split('+t+')[0] in self.datatmp[name]:
                    tag[i] = None
                elif is2pt and (self.is2pt is not True):
                    if tag[i].split('+')[self.zid] != self.is2pt:
                        tag[i] = None
        # add tags to the rows
        tag1 = []
        for i in range(len(tag)):
            tg = tag[i]
            if tg is None:
                continue
            tag1.append(tg)
            pz = tg.split('+t+')[0]
            if pz not in tab.index:
                tY = []
                for key in tag:
                    if key is None:
                        continue
                    if pz+'+t+' in key and (int(key.split('+')[-2]) not in tY):
                        tY.append(int(key.split('+')[-2]))
                #print("Range of tY is {:}".format(tY))
                try:
                    NTl = NT[pz]
                except:
                    NTl = NT
                if isinstance(NTl, int) is False:
                    NTl =  tab.loc[:,'NT'].values[-1]
                if pztX is None:
                    pztXl = None
                elif isinstance(pztX, dict):
                    pztXl = [list(pztX[pz])]
                else:
                    pztXl = [pztX]
                #print("Data is {:} {:} {:} {:} {:} {:}".format(NTl, tab.loc[:,'ndata'].values[-1], tab.loc[:,'ntrn'].values[-1], tab.loc[:,'nlbl'].values[-1], pztXl,tY, [pz] ))
                tab = tab.append(pds.DataFrame(data={'NT': NTl, 'ndata': tab.loc[:,'ndata'].values[-1], 'ntrn': tab.loc[:,'ntrn'].values[-1], 'nlbl': tab.loc[:,'nlbl'].values[-1], 
                                              'pztX': pztXl, 'tY': [list(tY)]}, index = [pz]))
       # del tab
        if dscale is None:
            dscale = 1.0
        for i in range(len(tag)):
            tg = tag[i]
            if tg is None:
                continue
            #if is2pt:
             #   print(tg)
            if isinstance(dscale, dict):
                scale = dscale[tg]
            else:
                try:
                    scale = dscale[i]
                except:
                    scale = dscale
            #print("scale = {:}".format(scale))
            pzt = tg.split('+t+')
            tri = pzt[1].split('+')
            pz = pzt[0]
            assert(pz in tab.index)
            t = int(tri[0])
            if tri[1] == 'R':
                ri = 0
            else:
                ri = 1
            if pz not in self.datatmp[name]:                
                self.datatmp[name][pz] = {} #{'R': list(ini), 'I': list(ini)}
            if tri[1] not in self.datatmp[name][pz]:
                self.datatmp[name][pz][tri[1]] = [[self.NA for n in range(tab.loc[pz, 'ndata'])] for m in range(tab.loc[pz, 'NT'])]
            if data is not None:
                self.datatmp[name][pz][tri[1]][t] = np.array(np.array(list(data[i]))*scale).tolist()
                continue
            if dtrn is not None:
                #print(list(dtrn[i]))
                self.datatmp[name][pz][tri[1]][t][:tab.loc[pz, 'ntrn']] = np.array(np.array(list(dtrn[i]))*scale).tolist()
            if dbc is not None:
                self.datatmp[name][pz][tri[1]][t][tab.loc[pz, 'ntrn']:tab.loc[pz, 'nlbl']] = np.array(np.array(list(dbc[i]))*scale).tolist()
            if dunlbl is not None:
                self.datatmp[name][pz][tri[1]][t][tab.loc[pz, 'nlbl']:] = np.array(np.array(list(dunlbl[i]))*scale).tolist()
        if self.ave:
            pzl = []
            for i in range(len(tag)):
                tg = tag[i]
                if tg is None:
                    continue
                pz = tg.split('+t+')[0]
                if pz in pzl:
                    continue
                pzl.append(pz)
                ntn = int(tab.loc[pz, 'NT']/self.nsrc)
                self.datatmpave[name][pz] = {}
                for ri in ['R', 'I']:
                    self.datatmpave[name][pz][ri] = np.array(self.datatmp[name][pz][ri][:ntn])
                    for i in range(1, self.nsrc):
                        self.datatmpave[name][pz][ri] += np.array(self.datatmp[name][pz][ri][i*ntn:(i+1)*ntn])
                    self.datatmpave[name][pz][ri] /= float(ntn)
                    if self.datatmpave[name][pz][ri].shape[1]!=495:
                        print("Ndata of {:}+pz{:}+{:} is {:}\n".format(name, pz, ri, self.datatmpave[name][pz][ri].shape[1]))
            del pzl
        self.wkrtable[indx] = pds.DataFrame(tab)
        if False: #for n in self.datatmp:
            for pz in self.datatmp[n]:
                for ri in self.datatmp[n][pz]:
                    for t in range(len(self.datatmp[n][pz][ri])):
                        print("{:}+{:}+t+{:}+{:}: {:}".format(n, pz, t, ri, self.datatmp[n][pz][ri][t]))
        del tab
        if True: #if (data is not None) or ((dtrn is not None) and (dbc is not None) and (dunlbl is not None)):
            self.add_mean(tag=tag1, pred=pred, indx=indx)
            self.add_cov(tag=tag1, pred=pred, indx=indx)
            #if pred is None:
             #   self.add_mean(tag=tag1, pred=True, indx=indx)
        return
    
    # Add the data mean to the list of features
    # tag: data id's 
    # data: corresponding data, None for using stored data
    # pred: None / True / False for calculating bias-corrected mean, direct predicted mean, and observed mean
    def add_mean(self, tag, data=None, pred=None, indx=None):
        if indx is None:
            indx = -1
        tab = self.wkrtable[indx]
        if pred is False:
            Yname = 'Yobs'
            name = 'Yobsmean'
        else:
            Yname = 'Ypdt'
            if pred is None:
                name = 'Ymean'
            else:
                name = 'Ypdtmean'
        if (data is None) and (Yname not in self.datatmp): #tab.columns):
            print("Error: Missing data for adding mean of data {:}".format(Yname))
            sys.exit(1)
        if pred is None:
            if (data is None) and ('Yobs' not in self.datatmp):
                print("Error: Missing observed data for adding mean of data {:}".format(Yname))
                sys.exit(1)
        if data is not None:
            if Yname in self.datatmp: #tab.columns:
                print("Warning: overwriting existing data!")
            self.add_data(tag, data=data, pred=pred, indx=indx)
        #dic = {'R': list(ini), 'I': list(ini)}
        if name+'R' not in tab.columns:
            tab.insert(len(tab.columns), name+'R', None)
            if pred is not None:
                tab.insert(len(tab.columns), name+'TrnR', None)
                tab.insert(len(tab.columns), name+'BCR', None)
                tab.insert(len(tab.columns), name+'AllR', None)
                tab.insert(len(tab.columns), name+'lblR', None)
        if name+'I' not in tab.columns:
            tab.insert(len(tab.columns), name+'I', None)
            if pred is not None:
                tab.insert(len(tab.columns), name+'TrnI', None)
                tab.insert(len(tab.columns), name+'BCI', None)
                tab.insert(len(tab.columns), name+'AllI', None)
                tab.insert(len(tab.columns), name+'lblI', None)
     #   for key in tab.index:
      #      tab.loc[key, name] = np.array([None for i in range(tab.loc[key, 'NT']*2)])
        for tg in tag:
            pz = tg.split('+t+')[0]
            if pz not in self.datatmp[Yname]:#if tab.loc[pz, Yname] is None:
                continue
            if pred is None:
                 if pz not in self.datatmp['Yobs']:#if tab.loc[pz,'Yobs'] is None:
                        continue
            tri = tg.split('+')
            t=int(tri[-2])
            if isinstance(tab.loc[pz, name+tri[-1]], list) is False: # is None:
                #tab.loc[pz, name] = pds.Series({'R': list(ini), 'I': list(ini)})
                tab[name+tri[-1]][pz] = [self.NA for i in range(tab.loc[pz, 'NT'])]
                if pred is not None:
                    tab[name+'Trn'+tri[-1]][pz] = [self.NA for i in range(tab.loc[pz, 'NT'])]
                    tab[name+'BC'+tri[-1]][pz] = [self.NA for i in range(tab.loc[pz, 'NT'])]
                    tab[name+'All'+tri[-1]][pz] = [self.NA for i in range(tab.loc[pz, 'NT'])]
                    tab[name+'lbl'+tri[-1]][pz] = [self.NA for i in range(tab.loc[pz, 'NT'])]
            if tri[-1] == 'R':
                ri = 0
            else:
                ri = 1
            if pred is None:
                tab[name+tri[-1]][pz][t] = np.nanmean(np.array(
                self.datatmp[Yname][pz][tri[-1]][t][tab.loc[pz,'nlbl']:])) + np.nanmean(np.array(
                self.datatmp['Yobs'][pz][tri[-1]][t][tab.loc[pz,'ntrn']:tab.loc[pz,'nlbl']]))-np.nanmean(np.array(
                self.datatmp[Yname][pz][tri[-1]][t][tab.loc[pz,'ntrn']:tab.loc[pz,'nlbl']]))
            else:
                tab[name+'lbl'+tri[-1]][pz][t] = np.nanmean(np.array(self.datatmp[Yname][pz][tri[-1]][t][:tab.loc[pz,'nlbl']]))
                tab[name+'All'+tri[-1]][pz][t] = np.nanmean(np.array(self.datatmp[Yname][pz][tri[-1]][t]))
                tab[name+tri[-1]][pz][t] = np.nanmean(np.array(self.datatmp[Yname][pz][tri[-1]][t][tab.loc[pz,'nlbl']:]))
                tab[name+'Trn'+tri[-1]][pz][t] = np.nanmean(np.array(self.datatmp[Yname][pz][tri[-1]][t][:tab.loc[pz,'ntrn']]))
                tab[name+'BC'+tri[-1]][pz][t] = np.nanmean(np.array(self.datatmp[Yname][pz][tri[-1]][t][tab.loc[pz,'ntrn']:tab.loc[pz,'nlbl']]))
        return
    
    # Add the covariance matrix to the list of data features 
    def add_cov(self, tag, data=None, pred=None, indx=None):
        if indx is None:
            indx = -1
        tab =  self.wkrtable[indx]
        if pred is True:
            name = 'ycovpdt'
            nameave = 'ycovavepdt'
            Yname = 'Ypdt'
        elif pred is False:
            name = 'ycovobs'
            nameave = 'ycovaveobs'
            Yname = 'Yobs'
        else:
            Yname = 'Ypdt'
            nameave = 'ycovave'
            name = 'ycov'
        if Yname not in self.datatmp: #tab.columns: 
            if data is None:
                print("Error: Missing data for generating covariance matrix!")
                sys.exit(1)
            self.add_data(tag, data=data, pred=pred, indx=indx)
        if name+'R' not in tab.columns:
            tab.insert(len(tab.columns), name+'R', None)
            if self.ave:
                tab.insert(len(tab.columns), nameave+'R', None)
        if name+'I' not in tab.columns:
            tab.insert(len(tab.columns), name+'I', None)
            if self.ave:
                tab.insert(len(tab.columns), nameave+'I', None)
        for key in tab.index:
            if key not in self.datatmp[Yname]:#if tab.loc[key, Yname] is None:
                continue
            if pred is None:
                if key not in self.datatmp['Yobs']: #if tab.loc[key, 'Yobs'] is None:
                    continue
            ntn = int(tab.loc[key, 'NT']/self.nsrc)
            if name == 'ycov':
                for ri in ['R', 'I']:
                    lst = list(self.datatmp['Ypdt'][key][ri])
                    lst.extend(self.datatmp['Yobs'][key][ri])
                    tab[name+ri][key] = np.cov(np.array(lst)) 
                    if self.DEBUG:
                        print('shape of ycov{:}: {:}'.format(ri, tab[name+ri][key].shape))
                    if self.ave is False:
                        del lst
                        continue
                    lsta = [ np.array(lst[i*ntn:(i+1)*ntn]) for i in range(2*self.nsrc)]
                    suma = [ lsta[0], lsta[self.nsrc] ]
                    for i in range(1, self.nsrc):
                        suma[0] = suma[0] + lsta[i]
                        suma[1] = suma[1] + lsta[i+self.nsrc]
                    suma[0] = suma[0] / float(self.nsrc)
                    suma[1] = suma[1] / float(self.nsrc)
                    lst = suma[0].tolist()
                    lst.extend(suma[1].tolist())
                    tab[nameave+ri][key] = np.cov(np.array(lst)) #np.cov(np.ndarray(shape=(2*self.nsrc, tab.loc[key, 'ndata']), buffer=suma))
                    del lst
                    del lsta
            else:
                for ri in ['R', 'I']:
                    lst = np.array(list(self.datatmp[Yname][key][ri]))
                    tab[name+ri][key] = np.cov(lst) 
                    if self.ave is False:
                        del lst
                        continue
                    lsta = [ np.array(lst[i*ntn:(i+1)*ntn]) for i in range(self.nsrc)]
                    suma = lsta[0]
                    for i in range(1, self.nsrc):
                        suma = suma + lsta[i]
                    suma = suma / float(self.nsrc)
                    tab[nameave+ri][key] = np.cov(suma)
                    del lst
                    del lsta
        #del self.wkrtable[indx]
        #self.wkrtable[indx] = tab
        if self.DEBUG:
            print(self.wkrtable[indx])
        return
                
    def apply_ffunc(self, func):
        if func not in self.afuncs:
            print()
            sys.exit(1)
        lyt = {}
        if 'pztY' in self.keys:
            for pzt in self.wkrtable[indx].loc['pztY']:
                lst = pzt.split('+')
                ky = lst[0]+'+'+lst[1]
                if ky not in lyt:
                    lyt[ky] = []
                lyt[ky].append(int(lst[2]))
        fit = {}
        if 'ycov' in self.keys and 'ymean' in self.keys:
            self.biascorrt_cov()
            ct = 0
            for ky in lyt:
                lt = 2*len(lyt[ky])
                fit[ky] = []
                for i in range(2):
                    mean = self.wkrtable[indx].loc['ymean'][ct:ct+lt:2]
                    cov =self.wkrtable[indx].loc['bcycov'][ct:ct+lt:2, ct:ct+lt:2]
                    fit[ky].append(lsq_fit(data=(lyt[ky], mean, cov), 
                                      prior=func[1], fcn=func[0]))
        self.wkrtable[-1].insert('fit.'+func, fit)
        return
    
    # Calculate the effective mass of 2pt correlators 
    # pred: True / False for predicted / observed data estimators
    # subset: 0: all data; 1: training subset; 2: bias correction subset; 3: unlabeled subset
    def effmass(self, tag, pred=None, subset=3, indx=None):
        if indx is None:
            indx = -1
        tab = self.wkrtable[indx]
        if pred is None:
            Yname = 'Y'
            cname0 = 'bcycov'
            cnameave0 = 'bcycovave'
        elif pred is True:
            Yname = 'Ypdt'
            cname0 = 'ycovpdt'
            cnameave0 = 'ycovavepdt'
        else:
            Yname = 'Yobs'
            cname0 = 'ycovobs'
            cnameave0 = 'ycovaveobs'
        name = Yname+'mean'
        if subset == 0:
            name = name+'All'
        elif subset == 1:
            name = name+'Trn'
        elif subset == 2:
            name = name+'BC'
        elif subset == 4:
            name = name+'lbl'
        if (Yname not in self.datatmp) and (name+'R' not in tab.columns) and (name+'I' not in tab.columns):
            print('Missing {:} data for effective mass calcualtions!'.format(Yname))
            sys.exit(1)
        if (name+'R' not in tab.columns) and (name+'I' not in tab.columns):
            self.add_mean(tag, data=None, pred=pred, indx=indx)
        if (cname0+'R' not in tab.columns):
            self.add_cov(tag, data=None, pred=pred, indx=indx)
        rname0 = 'effmass'
        rnameave0 = 'effmassave'
        if subset == 0:
            rname0 = rname0+'All'
            rnameave0 = rnameave0+'All'
        elif subset == 1:
            rname0 = rname0+'Trn'
            rnameave0 = rnameave0+'Trn'
        elif subset == 2:
            rname0 = rname0+'BC'
            rnameave0 = rnameave0+'BC'
        elif subset == 4:
            rname0 = rname0+'lbl'
            rnameave0 = rnameave0+'lbl'
        if pred is True:
            rname0 = rname0+'pdt'
            rnameave0 = rnameave0+'pdt'
        elif pred is False:
            rname0 = rname0+'obs'
            rnameave0 = rnameave0+'obs'
        if pred is None:
            if (cname0+'R' not in tab.columns) and (cname0+'I' not in tab.columns):
                self.biascorrt_cov(tag, indx)
            if self.checkBC:
                if ('effmassBCpdtR' not in tab.columns) and ('effmassBCpdtI' not in tab.columns):
                    self.effmass(tag, pred=True, subset=2, indx=indx)
                if ('effmassBCobsR' not in tab.columns) and ('effmassBCobsI' not in tab.columns):
                    self.effmass(tag, pred=False, subset=2, indx=indx)
            if ('effmasslblobsR' not in tab.columns) and ('effmasslblobsI' not in tab.columns):
                self.effmass(tag, pred=False, subset=4, indx=indx)
            if ('effmassAllobsR' not in tab.columns) and ('effmassAllobsI' not in tab.columns):
                self.effmass(tag, pred=False, subset=0, indx=indx)
        for ri in ['R', 'I']:
            if rname0+ri not in tab.columns:
                #lst = [[[self.NA, self.NA] for i in range(tab.loc[k, 'NT']-1)] for k in tab.index]
                tab.insert(len(tab.columns), rname0+ri, None)#lst)
                print("Add column {:}".format(rname0+ri))
                if self.ave:
                    tab.insert(len(tab.columns), rnameave0+ri, None)
                    print("Add column {:}".format(rnameave0+ri))
        name0 = name
        for tg in tag: 
            # skip 3pt data
            if '+T+' in tg:
                continue
            #if (self.is2pt is not True) and self.is2pt != tg.split('+')[1]:
            #    continue
            #print("Effmass tag is {:}".format(tg))
            pz = tg.split('+t+')[0]
            t = int(tg.split('+')[-2])
            if t >= tab.loc[pz,'NT']-1:
                continue
            ri = tg.split('+')[-1]
            rname = rname0+ri
            rnameave = rnameave0+ri
            name = name0+ri
            cname = cname0+ri
            cnameave = cnameave0+ri
            ntn = int(tab.loc[pz,'NT']/self.nsrc)
            ndata = tab.loc[pz,'ndata']-tab.loc[pz,'nlbl']
            if subset == 0:
                ndata = tab.loc[pz,'ndata']
            elif subset == 1:
                ndata = tab.loc[pz,'ntrn']
            elif subset == 2:
                ndata = tab.loc[pz,'nlbl'] - tab.loc[pz,'ntrn']
            elif subset == 4:
                ndata = tab.loc[pz,'nlbl']
            if isinstance(tab.loc[pz, rname], list) is False:
                #print("Generating effmass table {:} NT= {:}".format(pz, tab.loc[pz, 'NT']))
                tab[rname][pz] = [(self.NA, self.NA) for i in range(tab.loc[pz, 'NT']-1)]
                if self.ave:
                    tab[rnameave][pz] = [(self.NA, self.NA) for i in range(ntn-1)]
            if (tab.loc[pz, name][t] is self.NA) or (tab.loc[pz, name][t+1] is self.NA):
                continue
            try:
                Emean = math.log(tab.loc[pz,name][t] / tab.loc[pz,name][t+1])
            except:
                Emean = self.NA
            #print("Effmass at t={:} is {:}".format(t, Emean))
            if Emean == self.NA:
                continue
            try:
                Amean = tab.loc[pz,name][t] * math.exp(Emean*t)
                H = np.array([[0.,0.],[0.,0.]])
                for i in range(2):
                    sigma2 = tab.loc[pz,cname][t+i,t+i]
                    if self.DEBUG:
                        print(sigma2)
                        print(Emean)
                    H[0,0] += 2.*math.exp(-2.*Emean*float(t+i))/sigma2
                    H[0,1] += 2.*(tab.loc[pz,name][t+i]-2.*math.exp(-Emean*float(t+i))*Amean)*float(t+i)*math.exp(-Emean*float(t+i))/sigma2
                    H[1,1] += 2.*Amean*float(t+i)**2*math.exp(-Emean*float(t+i))*(2.*Amean*math.exp(-Emean*float(t+i))-tab.loc[pz,name][t+i])/sigma2
                H[1,0] = H[0,1]
                err = np.linalg.inv(H)
                if self.DEBUG:
                    print(err)
                tab[rname][pz][t] = (Emean, math.sqrt(err[1,1]/float(ndata)))
            except:
                tab[rname][pz][t] = (Emean, self.NA)
            if (self.ave is False) or (t >= ntn-1):
                continue
            # effmass averaged over sources
            ymean = [0., 0.]   
            nnn = 0
            #print("Nsrc is {:} {:}".format(self.nsrc, ntn))
            for i in range(self.nsrc):
                if (tab.loc[pz,name][t+i*ntn] + tab.loc[pz,name][t+i*ntn+1]) is not self.NA:
                    ymean = [ymean[j] + tab.loc[pz,name][t+j+i*ntn] for j in range(2)]
                    nnn += 1
            ymean[0] /= float(nnn)
            ymean[1] /= float(nnn)
            try:
                Emeanave = math.log(ymean[0] / ymean[1])
            except:
                Emeanave = self.NA
            if Emeanave == self.NA:
                continue
            try:
                Ameanave = ymean[0] * math.exp(Emeanave*float(t))
            except:
                tab[rnameave][pz][t] = (Emeanave, self.NA)
                continue
            try:
                H = np.array([[0.,0.],[0.,0.]]) 
                for i in range(2):
                    sigma2 = tab.loc[pz,cnameave][t+i,t+i]
                    if self.DEBUG:
                        print(sigma2)
                        print(Emeanave)
                    H[0,0] += 2.*math.exp(-2.*Emeanave*float(t+i))/sigma2
                    H[0,1] += 2.*(ymean[i]-2.*math.exp(-Emeanave*float(t+i))*Ameanave)*float(t+i)*math.exp(-Emeanave*float(t+i))/sigma2
                    H[1,1] += 2.*Ameanave*float(t+i)**2*math.exp(-Emeanave*float(t+i))*(2.*Ameanave*math.exp(-Emeanave*float(t+i))-ymean[i])/sigma2
                H[1,0] = H[0,1]
                err = np.linalg.inv(H)
                if self.DEBUG:
                    print(err)
                tab[rnameave][pz][t] = (Emeanave, math.sqrt(err[1,1]/float(ndata)))
            except:
                tab[rnameave][pz][t] = (Emeanave, self.NA)
        if self.DEBUG:
            print(self.wkrtable[indx])
        return

    # Calculate the ratio of 3pt and 2pt data
    # d2pt: 2pt data served as the denominator, None for using stored data
    # rtid: various ratios distinguished by the denorminator at: 
    #       t = t_sep/2 (type 0); t = t_3pt, t_sep - t_3pt (type 1)
    def ratio3ptn2pt(self, tag, d2pt=None, rtid=0, pred=None, subset = 3, indx=None):
        if (self.is2pt is True) or (self.is2pt is False):
            return
        if indx is None:
            indx = -1
        tab = self.wkrtable[indx]
        if pred is None:
            Yname = 'Ypdt'
            mname = 'Ymean'
        elif pred is True:
            Yname = 'Ypdt'
            mname = 'Ypdtmean'  
        else:
            Yname = 'Yobs'
            mname = 'Yobsmean'
        rname = 'Y3ptdv2pt'
        rnameave = 'Y3ptdv2ptave'
        if subset == 0:
            mname = mname+'All'
            rname = rname+'All'
            rnameave = rnameave+'All'
        elif subset == 1:
            mname = mname+'Trn'
            rname = rname+'Trn'
            rnameave = rnameave+'Trn'
        elif subset == 2:
            mname = mname+'BC'
            rname = rname+'BC'
            rnameave = rnameave+'BC'
        elif subset == 4:
            mname = mname+'lbl'
            rname = rname+'lbl'
            rnameave = rnameave+'lbl'
        if pred is True:
            rname = rname+'pdt'
            rnameave = rnameave+'pdt'
        elif pred is False:
            rname = rname+'obs'
            rnameave = rnameave+'obs'
        if (Yname not in self.datatmp) and (mname+'R' not in tab.index) and (mname+'I' not in tab.index):
            print("Error: Missing data for 3pt&2pt ratios!")
            sys.exit(1)
        if (mname+'R' not in tab.index) and (mname+'I' not in tab.index):
                self.add_mean(tag=tag, pred=pred, indx=indx)
        for ri in ['R', 'I']:
            if rname+ri not in tab.columns:
                tab.insert(len(tab.columns), rname+ri, None)
                if self.ave:
                    tab.insert(len(tab.columns), rnameave+ri, None)
            if pred is None:
                if ('Y3ptdv2ptlblobsR' not in tab.columns) and ('Y3ptdv2ptlblobsI' not in tab.columns):
                    self.ratio3ptn2pt(tag, pred=False, subset=4, indx=indx)
                if ('Y3ptdv2ptAllobsR' not in tab.columns) and ('Y3ptdv2ptAllobsI' not in tab.columns):
                    self.ratio3ptn2pt(tag, pred=False, subset=0, indx=indx)
        for tg in tag: 
            # skip 2pt data
            if '+T+' not in tg: #if self.is2pt == z:
                continue
            pz = tg.split('+t+')[0]
            if pz not in tab.index:
                continue
            try:
                o = pz.split('+')[self.oid]+'+'
            except:
                o = ''
            p = pz.split('+')[self.pid]
            z = pz.split('+')[self.zid]
            tsp = int(pz.split('+')[-1])
            t = int(tg.split('+')[-2])
            ntn = int(tab.loc[pz, 'NT']/self.nsrc)
            if t%ntn > tsp:
                continue
            ri = tg.split('+')[-1]
            ri2pt = 'R'
            #3pt data
            #find spectrum (local 2pt)
            p2pt = p
            if self.ratio_mp and (float(p) != 0.): 
                if float(p) > 0.:
                    p2pt = '-'+p
                else:
                    p2pt = p.split('-')[1]
            id2pt = o+p2pt+'+'+self.is2pt
            if rtid == 0:
                if d2pt is None:
                    if id2pt in tab.index:
                        print("Using stored data as denominator in 3pt&2pt ratios at p,z= {:} , {:}\n".format(p, z))
                        try:
                            d2ptn = self.datatmp[Yname][id2pt][ri2pt][int(tsp)]
                            if subset == 1:
                                d2ptn = self.datatmp[Yname][id2pt][ri2pt][int(tsp)][:tab.loc[id2pt, 'ntrn']]
                            elif subset == 2:
                                d2ptn = self.datatmp[Yname][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                            elif subset == 3:
                                d2ptn = self.datatmp[Yname][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'nlbl']:]
                            elif subset == 4:
                                d2ptn = self.datatmp[Yname][id2pt][ri2pt][int(tsp)][:tab.loc[id2pt, 'nlbl']]
                            if pred is not None:
                                d2ptm = d2ptn.mean()
                            else:
                                d2ptm = np.array(self.datatmp[Yname][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'nlbl']:]
                                                ).mean()+np.array(self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                                                 ).mean()-np.array(self.datatmp[Yname][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]).mean()
                        except:
                            print("Using observed 2pt data for 3pt&2pt ratio\n")
                            d2ptn = self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)]
                            if subset == 1:
                                d2ptn = self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)][:tab.loc[id2pt, 'ntrn']]
                            elif subset == 2:
                                d2ptn = self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                            elif subset == 3:
                                d2ptn = self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'nlbl']:]
                            elif subset == 4:
                                d2ptn = self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)][:tab.loc[id2pt, 'nlbl']]
                            d2ptm = np.array(d2ptn).mean()
                            #d2ptm = tab.loc[id2pt, 'Yobsmean'+ri2pt][int(tsp)]
                        if pred is None:
                            try:
                                jkbc2pt = self.datatmp[Yname][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                            except:
                                jkbc2pt = self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                        if self.ave and (t < ntn):
                            try:
                                d2ptnave = self.datatmpave[Yname][id2pt][ri2pt][int(tsp)]
                                if subset == 1:
                                    d2ptnave = self.datatmpave[Yname][id2pt][ri2pt][int(tsp)][:tab.loc[id2pt, 'ntrn']]
                                elif subset == 2:
                                    d2ptnave = self.datatmpave[Yname][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                elif subset == 3:
                                    d2ptnave = self.datatmpave[Yname][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'nlbl']:]
                                elif subset == 4:
                                    d2ptnave = self.datatmpave[Yname][id2pt][ri2pt][int(tsp)][:tab.loc[id2pt, 'nlbl']]
                                if pred is not None:
                                    d2ptmave = d2ptnave.mean()
                                else:
                                    d2ptmave = np.array(self.datatmpave[Yname][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'nlbl']:]
                                                ).mean()+np.array(self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                                                 ).mean()-np.array(self.datatmpave[Yname][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]).mean()
                            except:
                                d2ptnave = self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)]
                                if subset == 1:
                                    d2ptnave = self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)][:tab.loc[id2pt, 'ntrn']]
                                elif subset == 2:
                                    d2ptnave = self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                elif subset == 3:
                                    d2ptnave = self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'nlbl']:]
                                elif subset == 4:
                                    d2ptnave = self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)][:tab.loc[id2pt, 'nlbl']]
                                d2ptmave = d2ptnave.mean()
                            if pred is None:
                                jkbco2ptave = self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                try:
                                    jkbc2ptave = self.datatmpave[Yname][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                except:
                                    jkbc2ptave = self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                    else:
                        print("Error: Missing 2pt data for 3pt&2pt ratios at p= {:}".format(p))
                        sys.exit(1)
                else:
                    d2ptn = d2pt[ri2pt][int(tsp)]
                    d2ptm = np.array(d2ptn[int(tsp)]).mean()
                    if self.ave and (t < ntn):
                        d2ptnave = np.array(d2ptn)
                        d2ptmave = d2ptm
                        for i in range(1, self.nsrc):
                            d2ptnave = d2ptnave + np.array(d2pt[ri2pt][int(tsp)+i*ntn])
                        d2ptnave /= float(self.nsrc)    
                        d2ptmave = d2ptnave.mean()        
            elif rtid == 1:
                if d2pt is None:
                    if id2pt in tab.index:
                        print("Using stored data as denominator in 3pt&2pt ratios at p,z= {:} , {:}\n".format(p, z))
                        try:
                            d2ptn = [self.datatmp[Yname][id2pt][ri2pt][t], 
                                     self.datatmp[Yname][id2pt][ri2pt][tsp-t]]
                            d2ptml = [tab.loc[id2pt, mname+ri2pt][t], 
                                     tab.loc[id2pt, mname+ri2pt][tsp-t]]
                        except:
                            d2ptn = [self.datatmp['Yobs'][id2pt][ri2pt][t], 
                                     self.datatmp['Yobs'][id2pt][ri2pt][tsp-t]]
                            d2ptml = [tab.loc[id2pt, 'Yobsmean'+ri2pt][t], 
                                     tab.loc[id2pt, 'Yobsmean'+ri2pt][tsp-t]]
                        if pred is None:
                            try:
                                jkbc2pt = math.sqrt(np.array(self.datatmp[Yname][id2pt][ri2pt][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                                            )*np.array(self.datatmp[Yname][id2pt][ri2pt][tsp-t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]))
                            except:
                                jkbc2pt = math.sqrt(np.array(self.datatmp['Yobs'][id2pt][ri2pt][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                                            )*np.array(self.datatmp['Yobs'][id2pt][ri2pt][tsp-t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]))
                        if self.ave and (t < ntn): 
                            try:
                                d2ptnave = [self.datatmpave[Yname][id2pt][ri2pt][t], 
                                     self.datatmpave[Yname][id2pt][ri2pt][tsp-t]]
                                d2ptmlave = [np.array(d2ptnave[0][tab.loc[id2pt, 'nlbl']:]).mean()+
                                         np.array(self.datatmpave['Yobs'][id2pt][ri2pt][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]).mean()-
                                         np.array(d2ptnave[0][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]).mean(), 
                                         np.array(d2ptnave[1][tab.loc[id2pt, 'nlbl']:]).mean()+
                                         np.array(self.datatmpave['Yobs'][id2pt][ri2pt][tsp-t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]).mean()-
                                         np.array(d2ptnave[1][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]).mean()]
                            except:
                                if subset == 0:
                                    d2ptnave = [self.datatmpave['Yobs'][id2pt][ri2pt][t], 
                                         self.datatmpave['Yobs'][id2pt][ri2pt][tsp-t]]
                                elif subset == 1:
                                    d2ptnave = [self.datatmpave['Yobs'][id2pt][ri2pt][t][:tab.loc[id2pt, 'ntrn']], 
                                         self.datatmpave['Yobs'][id2pt][ri2pt][tsp-t][:tab.loc[id2pt, 'ntrn']]]
                                elif subset == 2:
                                    d2ptnave = [self.datatmpave['Yobs'][id2pt][ri2pt][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']], 
                                         self.datatmpave['Yobs'][id2pt][ri2pt][tsp-t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]]
                                else:
                                    d2ptnave = [self.datatmpave['Yobs'][id2pt][ri2pt][t][tab.loc[id2pt, 'nlbl']:], 
                                         self.datatmpave['Yobs'][id2pt][ri2pt][tsp-t][tab.loc[id2pt, 'nlbl']:]]
                                d2ptmlave = [d2ptnave[0].mean(), d2ptnave[1].mean()]
                            if pred is None:
                                jkbco2ptave = math.sqrt(np.array(self.datatmpave['Yobs'][id2pt][ri2pt][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                                            )*np.array(self.datatmpave['Yobs'][id2pt][ri2pt][tsp-t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]))
                                try:
                                    jkbc2ptave = math.sqrt(np.array(self.datatmpave[Yname][id2pt][ri2pt][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                                            )*np.array(self.datatmpave[Yname][id2pt][ri2pt][tsp-t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]))
                                except:
                                    jkbc2ptave = math.sqrt(np.array(self.datatmpave['Yobs'][id2pt][ri2pt][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                                            )*np.array(self.datatmpave['Yobs'][id2pt][ri2pt][tsp-t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]))
                    else:
                        print("Error: Missing 2pt data for 3pt&2pt ratios at p= {:}".format(p))
                        sys.exit(1)
                else:
                    d2ptn = [d2pt[ri2pt][t], d2pt[ri2pt][tsp-t]]
                    d2ptml = [np.array(d2ptn[0]).mean(), np.array(d2ptn[1]).mean()]
                    # FIXME! 
                    #if t < ntn: 
                    # ...
                d2ptm = math.sqrt(d2ptml[0]*d2ptml[1])
                if self.ave and (t < ntn):
                    d2ptmave = math.sqrt(d2ptmlave[0]*d2ptmlave[1])
            else:
                print("Error: unsupported ratio type {:}".format(rtid))
                sys.exit(1)
            name = rname+ri
            if isinstance(tab.loc[pz, name], list) is False: 
                tab[name][pz] = [(self.NA, self.NA) for i in range(tab.loc[pz, 'NT'])]
            #print("d2ptn is {:}".format(d2ptn))
            d3ptn = self.datatmp[Yname][pz][ri][t]
            if subset == 1:
                d3ptn = self.datatmp[Yname][pz][ri][t][:tab.loc[id2pt, 'ntrn']]
            elif subset == 2:
                d3ptn = self.datatmp[Yname][pz][ri][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
            elif subset == 3:
                d3ptn = self.datatmp[Yname][pz][ri][t][tab.loc[id2pt, 'nlbl']:]
            elif subset == 4:
                d3ptn = self.datatmp[Yname][pz][ri][t][:tab.loc[id2pt, 'nlbl']]
            if pred is not None:
                d3ptm = np.array(d3ptn).mean()
            else:
                d3ptm = np.array(self.datatmp[Yname][pz][ri][t][tab.loc[id2pt, 'nlbl']:]
                                ).mean()+np.array(self.datatmp['Yobs'][pz][ri][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                                 ).mean()-np.array(self.datatmp[Yname][pz][ri][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]).mean()
            if self.ave and (t < ntn):
                d3ptnave = self.datatmpave[Yname][pz][ri][t]
                if subset == 1:
                    d3ptnave = self.datatmpave[Yname][pz][ri][t][:tab.loc[id2pt, 'ntrn']]
                elif subset == 2:
                    d3ptnave = self.datatmpave[Yname][pz][ri][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                elif subset == 3:
                    d3ptnave = self.datatmpave[Yname][pz][ri][t][tab.loc[id2pt, 'nlbl']:]
                elif subset == 4:
                    d3ptnave = self.datatmpave[Yname][pz][ri][t][:tab.loc[id2pt, 'nlbl']]
                if pred is not None:
                    d3ptmave = d3ptnave.mean()
                else:
                    d3ptmave = np.array(self.datatmpave[Yname][pz][ri][t][tab.loc[id2pt, 'nlbl']:]
                                ).mean()+np.array(self.datatmpave['Yobs'][pz][ri][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                                 ).mean()-np.array(self.datatmpave[Yname][pz][ri][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]).mean()
            if pred is None:
                jkbc3pt = self.datatmp[Yname][pz][ri][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                jkbco3pt = self.datatmp['Yobs'][pz][ri][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                o3ptn = self.datatmp['Yobs'][pz][ri][t]
                if subset == 1:
                    o3ptn = self.datatmp['Yobs'][pz][ri][t][:tab.loc[id2pt, 'ntrn']]
                elif subset == 2:
                    o3ptn = self.datatmp['Yobs'][pz][ri][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                elif subset == 3:
                    o3ptn = self.datatmp['Yobs'][pz][ri][t][tab.loc[id2pt, 'nlbl']:]
                elif subset == 4:
                    o3ptn = self.datatmp['Yobs'][pz][ri][t][:tab.loc[id2pt, 'nlbl']]
                if self.ave and (t < ntn):
                    o3ptnave = self.datatmpave['Yobs'][pz][ri][t]
                    if subset == 1:
                        o3ptnave = self.datatmpave['Yobs'][pz][ri][t][:tab.loc[id2pt, 'ntrn']]
                    elif subset == 2:
                        o3ptnave = self.datatmpave['Yobs'][pz][ri][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                    elif subset == 3:
                        o3ptnave = self.datatmpave['Yobs'][pz][ri][t][tab.loc[id2pt, 'nlbl']:]
                    elif subset == 4:
                        o3ptnave = self.datatmpave['Yobs'][pz][ri][t][:tab.loc[id2pt, 'nlbl']]
                if rtid == 0:
                    o2ptn = self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)]
                    if subset == 1:
                        o2ptn = self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)][:tab.loc[id2pt, 'ntrn']]
                    elif subset == 2:
                        o2ptn = self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                    elif subset == 3:
                        o2ptn = self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'nlbl']:]
                    elif subset == 4:
                        o2ptn = self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)][:tab.loc[id2pt, 'nlbl']]
                    if self.ave and (t < ntn):
                        o2ptnave = self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)]
                        if subset == 1:
                            o2ptnave = self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)][:tab.loc[id2pt, 'ntrn']]
                        elif subset == 2:
                            o2ptnave = self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                        elif subset == 3:
                            o2ptnave = self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'nlbl']:]
                        elif subset == 4:
                            o2ptnave = self.datatmpave['Yobs'][id2pt][ri2pt][int(tsp)][:tab.loc[id2pt, 'nlbl']]
                    extcov = np.cov(np.array([d3ptn, d2ptn, 
                                              o3ptn, 
                                              o2ptn]))
                    jkbco2pt = self.datatmp['Yobs'][id2pt][ri2pt][int(tsp)][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                elif rtid == 1:
                    jkbco2pt = math.sqrt(np.array(self.datatmp['Yobs'][id2pt][ri2pt][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                                                 )*np.array(self.datatmp['Yobs'][id2pt][ri2pt][tsp-t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]))
                    extcov = np.cov(np.array([d3ptn, d2ptn[0], d2ptn[1], 
                                              o3ptn, 
                                              self.datatmp['Yobs'][id2pt][ri2pt][t], 
                                             self.datatmp['Yobs'][id2pt][ri2pt][tsp-t]]))
                assert(list(extcov.shape)==[4+rtid*2,4+rtid*2])
                cov = (2.*extcov[:2+rtid, :2+rtid] + extcov[2+rtid:, 2+rtid:] - extcov[:2+rtid,2+rtid:] - extcov[2+rtid:,:2+rtid])
                      #   -np.matmul(extcov[:2,2:], extcov[2:,:2])-np.matmul(extcov[2:,:2], extcov[:2,2:]))
            else:
                if rtid == 0:
                    #print("3pt data is ({:}) {:} \n 2pt data is ({:}) {:}".format(len(self.datatmp[Yname][pz][ri][t]), self.datatmp[Yname][pz][ri][t], len(d2ptn), d2ptn))
                    #print("Shape of d3ptn: {:} d2ptn: {:}\n\n".format(np.array(d3ptn).shape, np.array(d2ptn).shape))
                    cov = np.cov(np.array([d3ptn, d2ptn]))
                elif rtid == 1:
                    cov = np.cov(np.array([d3ptn, d2ptn[0], d2ptn[1]]))
            if rtid == 0:
                df = np.ndarray(shape=(1, 2), buffer=np.array([1.0/d2ptm, -tab.loc[pz, mname+ri][t]/d2ptm**2]))
            elif rtid == 1:
                df = np.ndarray(shape=(1, 3), buffer=np.array([1.0, -.5*tab.loc[pz, mname+ri][t]/d2ptml[0], 
                                                               -.5*tab.loc[pz, mname+ri][t]/d2ptml[1]])/d2ptm)
            err = np.matmul(np.matmul(df, cov), df.T)[0,0]/math.sqrt(float(tab.loc[pz,'ndata']-tab.loc[pz,'nlbl']))
            if rtid == 0:
                jkrt = [(d3ptm*float(len(d3ptn))-d3ptn[i])/(d2ptm*float(len(d3ptn))-d2ptn[i]) for i in range(len(d3ptn))]
            else:
                jkrt = [(d3ptm*float(len(d3ptn))-d3ptn[i])/(d2ptm*float(len(d3ptn))-math.sqrt(d2ptn[0][i]*d2ptn[1][i])) for i in range(len(d3ptn))]
            jkerr = np.array(jkrt).std(ddof=1)*float(len(d3ptn)-1)/math.sqrt(float(len(d3ptn)))
            # bias correction jk error
            if (pred is None) and (d2pt is None):
                jkbc2ptm = np.array(jkbc2pt).sum()
                jkbco2ptm = np.array(jkbco2pt).sum()
                jkbc3ptm = np.array(jkbc3pt).sum()
                jkbco3ptm = np.array(jkbco3pt).sum()
                jkbcerr = np.array([(jkbc3ptm-jkbc3pt[i])/(jkbc2ptm-jkbc2pt[i])-(jkbco3ptm-jkbco3pt[i])/(jkbco2ptm-jkbco2pt[i]) for i in range(len(jkbc3pt))]
                                  ).std(ddof=1)*float(len(jkbc3pt)-1)/math.sqrt(float(len(jkbc3pt)))
            else:
                jkbcerr = 0.0
            tab[name][pz][t] = (d3ptm/d2ptm, err, jkerr, jkbcerr)
            if (t >= ntn) or (self.ave is False):
                continue
            name = rnameave+ri
            if isinstance(tab.loc[pz, name], list) is False: 
                tab[name][pz] = [(self.NA, self.NA) for i in range(tab.loc[pz, 'NT'])]
            if pred is None:
                jkbc3ptave = self.datatmpave[Yname][pz][ri][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                jkbco3ptave = self.datatmpave['Yobs'][pz][ri][t][tab.loc[id2pt, 'ntrn']:tab.loc[id2pt, 'nlbl']]
                if rtid == 0:
                    extcov = np.cov(np.array([d3ptnave, d2ptnave, #self.datatmpave[Yname][pz][ri][t], d2ptnave, 
                                              o3ptnave, #self.datatmpave['Yobs'][pz][ri][t], 
                                              o2ptnave ]))#self.datatmpave['Yobs'][id2pt][ri][int(tsp)]]))
                elif rtid == 1:
                    extcov = np.cov(np.array([self.datatmpave[Yname][pz][ri][t], d2ptnave[0], d2ptnave[1],
                                              self.datatmpave['Yobs'][id2pt][ri][t], 
                                             self.datatmpave['Yobs'][id2pt][ri][tsp-t]]))
                assert(list(extcov.shape)==[4+rtid*2,4+rtid*2])
                cov = (2.*extcov[:2+rtid, :2+rtid] + extcov[2+rtid:, 2+rtid:] - extcov[:2+rtid,2+rtid:] - extcov[2+rtid:,:2+rtid])
            else:
                if rtid == 0:
                    print("shape of 3pt is {:} 2pt is {:}\n\n".format(len(d3ptnave), len(d2ptnave)))
                    cov = np.cov(np.array([d3ptnave, d2ptnave])) #self.datatmpave[Yname][pz][ri][t], d2ptnave]))
                elif rtid == 1:
                    cov = np.cov(np.array([d3ptnave, d2ptnave[0], d2ptnave[1]]))
            ave3pt = tab.loc[pz, mname+ri][t]
            for i in range(1, self.nsrc):
                ave3pt += tab.loc[pz, mname+ri][t+i*ntn]
            ave3pt /= float(self.nsrc)
            if rtid == 0:
                df = np.ndarray(shape=(1, 2), buffer=np.array([1.0/d2ptmave, -d3ptmave/d2ptmave**2]))
            elif rtid == 1:
                df = np.ndarray(shape=(1, 3), buffer=np.array([1.0, -.5*d3ptmave/d2ptmlave[0], 
                                                               -.5*d3ptmave/d2ptmlave[1]])/d2ptmave)
            err = np.matmul(np.matmul(df, cov), df.T)[0,0]/math.sqrt(float(tab.loc[pz,'ndata']-tab.loc[pz,'nlbl']))
            if rtid == 0:
                jkrt = [(np.array(d3ptnave).sum()-d3ptnave[i])/(np.array(d2ptnave).sum()-d2ptnave[i]) for i in range(len(d2ptnave))]
            else:
                jkrt = [(np.array(d3ptnave).sum()-d3ptnave[i])/(d2ptmave*float(len(d2ptnave[0]))-math.sqrt(d2ptnave[0][i]*d2ptnave[1][i])) for i in range(len(d3ptn))]
            jkerr = np.array(jkrt).std(ddof=1)*float(len(jkrt)-1)/math.sqrt(float(len(jkrt)))
            # bias correction jk error
            if (pred is None) and (d2pt is None):
                jkbc2ptm = np.array(jkbc2ptave).sum()
                jkbco2ptm = np.array(jkbco2ptave).sum()
                jkbc3ptm = np.array(jkbc3ptave).sum()
                jkbco3ptm = np.array(jkbco3ptave).sum()
                jkbcerr = np.array([(jkbc3ptm-jkbc3ptave[i])/(jkbc2ptm-jkbc2ptave[i])-(jkbco3ptm-jkbco3ptave[i])/(jkbco2ptm-jkbco2ptave[i]) for i in range(len(jkbc3ptave))]
                                  ).std(ddof=1)*float(len(jkbc3ptave)-1)/math.sqrt(float(len(jkbc3ptave)))
            else:
                jkbcerr = 0.0
            tab[name][pz][t] = (d3ptmave/d2ptmave, err, jkerr, jkbcerr)
        return
    
    # Print effective masses
    # pf: Output data filename
    # ppf: Output plot filename
    def print_effmass(self, indx=None, pf = None, ppf=None):
        if indx is None:
            idx = -1
        elif isinstance(indx, str):
            idx = 0
        else:
            idx = indx
        keylist = ['effmass', 'effmassTrn', 'effmassBC']
        if self.ave:
            keylist.extend(['effmassave', 'effmassaveTrn', 'effmassaveBC'])
        for key in keylist:
            ave = ''
            if 'ave' in key:
                ave = ' averaged over {:} sources'.format(self.nsrc)
            sfx = ''
            if 'Trn' in key:
                sfx = ' (Training subset)'
            if 'BC' in key:
                sfx = ' (Bias Correction subset)'
            name = {'main': key, 'description': 'Effective masses'+ave+sfx, 'complex': True}
            data = []
            if key+'R' in self.wkrtable[idx].columns:
                data.append(self.print_predTF(name, None, 0, indx, pf))
            if key+'obsR' in self.wkrtable[idx].columns:
                print("Add effmassobsR\n")
                data.append(self.print_predTF(name, False, 0, indx, pf))
            if key+'pdtR' in self.wkrtable[idx].columns:
                data.append(self.print_predTF(name, True, 0, indx, pf))
            if len(data) > 0:
                self.savefig(name, data, self.pltcid, indx, ppf)
            del data
        return
    
    def print_effmassv2(self, indx=None, pf = None, ppf=None):
        if indx is None:
            idx = -1
        elif isinstance(indx, str):
            idx = 0
        else:
            idx = indx
        keylist = [['effmass', 'effmasslbl', 'effmassAll']]
        if self.ave:
            keylist.append(['effmassave', 'effmassavelbl', 'effmassaveAll'])
        for keys in keylist:
            data = []
            for key in keys:
                ave = ''
                if 'ave' in key:
                    ave = ' averaged over {:} sources'.format(self.nsrc)
                sfx = ''
                if 'Trn' in key:
                    sfx = ' (Training subset)'
                if 'BC' in key:
                    sfx = ' (Bias Correction subset)'
                if 'lbl' in key:
                    sfx = ' (Labeled subset)'
                if 'All' in key:
                    sfx = ' (All data)'
                name = {'main': key, 'description': 'Effective masses'+ave+sfx, 'complex': True}
                #data = []
                if ('lbl' in key) or ('All' in key):
                    data.append(self.print_predTF(name, False, 0, indx, pf, 'v2'))
                else:
                    data.append(self.print_predTF(name, None, 0, indx, pf, 'v2'))
            if len(data) > 0:
                self.savefig(name, data, self.pltcid, indx, ppf, 'v2')
            del data
        return
    
    # Print ratio of 3pt and 2pt data
    def print_ratio3ptn2pt(self, indx=None, pf = None, ppf=None):
        if indx is None:
            indx = -1
        elif isinstance(indx, str):
            idx = 0
        else:
            idx = indx
        keylist = ['Y3ptdv2pt']
        if self.ave:
            keylist.append('Y3ptdv2ptave')
        #data = []
        for key in keylist:
            ave = ''
            if 'ave' in key:
                ave = ' averaged over {:} sources'.format(self.nsrc)
            sfx = ''
            if 'Trn' in key:
                sfx = ' (Training subset)'
            if 'BC' in key:
                sfx = ' (Bias Correction subset)'
            if 'lbl' in key:
                sfx = ' (Labeled subset)'
            if 'All' in key:
                sfx = ' (All data)'
            name = {'main': key, 'description': 'Ratio of 3pt & 2pt correlators'+ave+sfx, 'complex': True}
            data = []
            if key+'R' in self.wkrtable[idx].columns:
                data.append(self.print_predTF(name, None, 1, indx, pf))
            if key+'obsR' in self.wkrtable[idx].columns:
                data.append(self.print_predTF(name, False, 1, indx, pf))
            if key+'pdtR' in self.wkrtable[idx].columns:
                data.append(self.print_predTF(name, True, 1, indx, pf))
            self.savefig(name, data, self.pltcid, indx, ppf)
            del data
        return
    
    def print_ratio3ptn2ptv2(self, indx=None, pf = None, ppf=None):
        if indx is None:
            indx = -1
        elif isinstance(indx, str):
            idx = 0
        else:
            idx = indx
        keylist = [['Y3ptdv2ptAll', 'Y3ptdv2ptTrn', 'Y3ptdv2ptBC']]#, 'Y3ptdv2ptAll']]
        if self.ave:
            keylist.append(['Y3ptdv2ptave', 'Y3ptdv2ptavelbl'])#, 'Y3ptdv2ptaveAll'])
        #data = []
        for keys in keylist:
            data = []
            panel = []
            for key in keys:
                ave = ''
                if 'ave' in key:
                    ave = ' averaged over {:} sources'.format(self.nsrc)
                sfx = ''
                if 'Trn' in key:
                    sfx = ' (Training subset)'
                    if 'Trn' not in panel:
                        panel.append('Trn')
                if 'BC' in key:
                    sfx = ' (Bias Correction subset)'
                    if 'BC' not in panel:
                        panel.append('BC')
                if 'lbl' in key:
                    sfx = ' (Labeled subset)'
                    if 'lbl' not in panel:
                        panel.append('lbl')
                if 'All' in key:
                    sfx = ' (All data)'
                    if 'All' not in panel:
                        panel.append('All')
                name = {'main': key, 'description': 'Ratio of 3pt & 2pt correlators'+ave+sfx, 'complex': True}
                if True:#('lbl' in key) or ('All' in key):
                    data.append(self.print_predTF(name, False, 1, indx, pf, 'v2'))
                else:
                    data.append(self.print_predTF(name, None, 1, indx, pf, 'v2'))
            self.savefig(name, data, self.pltcid, indx, ppf, 'v2', panel)
            del data
            del panel
        return
        
    
    def print_predTF(self, name, pred, errid, indx, pf, version = None):
        if name['complex']: 
            ril = [('R', ' (Real)'), ('I', ' (Imag)')]
        else:
            ril = [('', '')]
        data = {}
        for ri in ril:
            data[ri[0]] = self.print_predTFRI(ri, name, pred, errid, indx, pf, version)
        return data
    
    def savefig(self, name, data, pltcid, indx, ppf, version = 'v1', panel=None):
        if name['complex']: 
            ril = [('R', ' (Real)'), ('I', ' (Imag)')]
        else:
            ril = [('', '')]
        for ri in ril:
            self.savefigRI(ri, name, data, pltcid, indx, ppf, version, panel)
        return
        
    def print_predTFRI(self, ri, name, pred, errid, indx, pf, version):
        if pred is None:
            rname = name['main']+ri[0]
            rnamenbcp = name['main']+'BCpdt'+ri[0]
            rnamenbco = name['main']+'BCobs'+ri[0]
            rkey = ' BC-Predicted'
            surfix = '+bcpred'
            offset = 0.2
        elif pred is True:
            rname = name['main']+'pdt'+ri[0]
            rkey = ' Predicted'
            surfix = '+pred'
            offset = -0.2
        else:
            rname = name['main']+'obs'+ri[0]
            rkey = ' Observed'
            surfix = '+obsv'
        if version == 'v2':
            surfix = surfix+name['main']
        offset = 0.
        print("Print name {:}".format(rname))
        if pred is None and self.checkBC:
            form = "\n {:}{:}{:}: \n   key    p    z    t    mean \t err \t BC-err \n"
        else:
            form = "\n {:}{:}{:}: \n   key    p    z    t    mean \t err \n"
            print(form.format(name['description'], ri[1], rkey))
        if pf is not None: 
            pf.write(form.format(name['description'], ri[1], rkey))
        if indx is None:
            indx = -1
        if isinstance(indx, str):
            em = {}
            if pred is None and self.checkBC:
                bcem = {}
            plt = pds.DataFrame()
            i = 0
            for tab in self.wkrtable:
                print("gathering sample {:}".format(i))
                i += 1
                for key in tab.index:
                    if isinstance(tab.loc[key, rname], list):# is not None:
                        try:
                            em[key].append(tab.loc[key, rname])
                        except:
                            em[key] = [tab.loc[key, rname]]
                        if pred is None and self.checkBC:
                            try:
                                bcem[key].append(np.array(tab.loc[key, rnamenbco])[:,0]-np.array(tab.loc[key, rnamenbcp])[:,0])
                            except:
                                bcem[key] = [np.array(tab.loc[key, rnamenbco])[:,0]-np.array(tab.loc[key, rnamenbcp])[:,0]]
            #print(em)
            yl = {}
            errl = {}
            tl = {}
            if pred is None and self.checkBC:
                bcerrl = {}
            for key in em:
                #self.pltcid += 1
                keyn = key+surfix
                yl[keyn] = []
                errl[keyn] = []
                tl[keyn] = []
                if pred is None and self.checkBC:
                    bcerrl[keyn] = []
                #if em[key][0] is None:
                 #   continue
                for t in range(len(em[key][0])):
                    if em[key][0][t][0] is self.NA:
                        continue
                    ema = np.array([em[key][i][t][0] for i in range(len(em[key]))])
                    #print("{:} on samples: {:}".format(key, ema))
                    emm = np.nanmean(np.array(ema[1:]))
                    #ems = np.nanstd(np.array(ema[1:]))#/math.sqrt(float(len(ema))-1.0)
                    eme = np.array([em[key][i][t][1] for i in range(len(em[key]))])
                    try:
                        jkeme = np.array([em[key][i][t][2] for i in range(len(em[key]))])
                        jkbceme = np.array([em[key][i][t][3] for i in range(len(em[key]))])
                    except:
                        jkeme = np.array([0 for i in range(len(em[key]))])
                        jkbceme = np.array([0 for i in range(len(em[key]))])
                    ems = np.nanmean(np.array(eme[1:]))
                    std = eme[0]
                    if pred is None and self.checkBC:
                        bcema = np.array([bcem[key][i][t] for i in range(len(bcem[key]))])
                        bcerr = np.nanstd(np.array(bcema[1:]), ddof=1)
                        print("bias correction of {:} on samples: ( {:} ) {:}".format(key, bcerr, bcema))
                        bcerrl[keyn].append(bcerr)
                    if indx == 'JK':
                        #emm = float(len(ema)-1.)*ema[0]-(float(len(ema))-2.)*emm
                        #ems = float(len(eme)-1.)*eme[0]-(float(len(eme))-2.)*ems
                        std = np.nanstd(np.array(ema[1:]), ddof=1)*float(len(ema)-2.)/math.sqrt(float(len(ema)-1.))
                        std = math.sqrt(jkeme[1:].mean()**2+jkbceme[1:].mean()**2+std**2)
                    elif indx == 'BS':
                        emm = 2.*ema[0] - emm
                        std = np.nanstd(np.array(ema[1:]), ddof=1)
                        if eme[0] is not self.NA:
                            ems = 2.*eme[0] - ems
                    #std = math.sqrt(jkeme[1:].mean()**2+jkbceme[1:].mean()**2+std**2)
                    try:
                        o = key.split('+')[self.oid]
                    except:
                        o = ''
                    if pred is None and self.checkBC:
                        form = "{:}    {:}    {:}    {:}    {:} \t {:}  /  {:} \t {:}".format(o, key.split('+')[self.pid], key.split('+')[self.zid], 
                                                                           t, emm, ems, std, bcerr)
                    else:
                        form = "{:}    {:}    {:}    {:}    {:} \t {:}  /  {:}".format(o, key.split('+')[self.pid], key.split('+')[self.zid], 
                                                                           t, emm, ems, std)
                    print(form)
                    if emm is self.NA:
                        del ema
                        del eme
                        if pred is None and self.checkBC:
                            del bcema
                        continue
                    if pf is not None:
                        pf.write(form+'\n')
                    if pred is None:
                        plt.insert(len(plt.columns), key+'+t'+str(t)+'.bcpred', ema.tolist())
                    elif pred is True:
                        plt.insert(len(plt.columns), key+'+t'+str(t)+'.pred', ema.tolist())
                    else:
                        plt.insert(len(plt.columns), key+'+t'+str(t), ema.tolist())
                    yl[keyn].append(emm)
                    if errid == 0:
                        errl[keyn].append(ems)
                    else:
                        errl[keyn].append(std)
                    tl[keyn].append(float(t)+offset)
                    del ema
                    del eme
                    if pred is None and self.checkBC:
                        del bcema
            if pred is None and self.checkBC:
                return {'x': tl, 'y': yl, 'yerr': errl, 'BCyerr': bcerrl}#plt
            return {'x': tl, 'y': yl, 'yerr': errl}
            #if ppf is not None:
            #Dataframe.boxplot(None, data=plt, title=name['description']+ri[1]+rkey, out=ppf)
            #Dataframe.errbarplot(tl, yl, errl, self.pltcid, name['description']+ri[1]+rkey, ppf)
        else:
            tab = self.wkrtable[indx]
            for key in tab.index:
                if tab.loc[key, rname] is not None:
                    if ppf is not None:
                        tl = []
                    for t in range(len(tab.loc[key, rname])):
                        if tab.loc[key, rname][t][0] != self.NA:
                            try:
                                o = key.split('+')[self.oid]
                            except:
                                o = ''
                            form = "{:}    {:}    {:}    {:}    {:} \t {:}\n".format(o, key.split('+')[self.pid], key.split('+')[self.zid], t, 
                                                                      tab.loc[key, rname][t][0], 
                                                                      tab.loc[key, rname][t][1])
                            print(form)
                            if pf is not None:
                                pf.write(form)
                            if ppf is not None:
                                tl.append(t)
                    y = [tab.loc[key, rname][t][0] for t in tl]
                    yerr = [tab.loc[key, rname][t][1] for t in tl]
                return tl, y, yerr
                    #Dataframe.errbarplot(tl, y, yerr, self.pltcid, name['description']+ri[1]+rkey, ppf)        
        return
    
    def savefigRI(self, ri, name, data, pltcid, indx, ppf, version, panel=None):
        panel = None
        #if version == 'v2':
         #   panel = ['lbl', 'All', 'bcpred']
        if True:
            if isinstance(indx, str):
                #print(data)
                df = {}
                for i in range(len(data)):
                    for key in data[i][ri[0]]:
                        try:
                            df[key].update(data[i][ri[0]][key])
                        except:
                            df[key] = data[i][ri[0]][key]
                if False:
                    df = data[0][ri[0]]
                    print(df)
                    for key in df:
                        for i in range(1, len(data)):
                            if key in data[i][ri[0]]:
                                df[key].update(data[i][ri[0]][key])
                print(df)
                if False: #for i in range(1, len(data)):
                    for col in data[i][ri[0]].columns:
                        df.insert(len(df.columns), col, data[i][ri[0]].loc[:, col])
                #Dataframe.boxplot(None, data=df, title=name['description']+ri[1], out=ppf)
                self.pltcid = Dataframe.errbarplot(None, data = df, title=name['description']+ri[1], clrid = self.pltcid, out = ppf, panel = panel)
            else:
                tl, y, err = data[0][ri[0]]
                for i in range(1, len(data)):
                    tln, yn, errn = data[i][ri[0]]
                    tl.extend(tln)
                    y.extend(yn)
                    err.extend(errn)
                Dataframe.errbarplot1(tl, y, yerr, pltcid, name['description']+ri[1], ppf) 
            return
    

        
    
    
    
    
    
# Maybe it's not needed     
class PDF_Functions:
    def __init__(self, nexp):
        self.nexp = nexp
        self.par = {'A': None, 'E': None, 'V': None, 'Vi': None}
        self.prior = dict(self.par)
        self.prop2prop = PDF_Functions.twopt(self)
    class twopt:
        def __init__(self, base):
            self._base = base
            #self.base.func = self.func()
            self.x = None
            self.y = None
        def Y(self):
            fy = []
            for x in self.x:
                y = 0.
                for n in self._base.nexp:
                    y += self._base.par['A'][n]*math.exp(-self._base.par['E'][n]*x)
                    #y.append(self.base.par['A']*math.exp(-self.base.par['E']*x))
                fy.append(y)
            return fy
        def dYd(self):
            fy = dict(self._base.par)
            for k in fy:
                fy[k] = [0. for i in range(self._base.nexp)]
            for x in self.x:
                for n in self.base.nexp:
                    fy['A'][n] += math.exp(-self._base.par['E'][n]*x)
                    fy['E'][n] += -x*self._base.par['A'][n]*math.exp(-self._base.par['E'][n]*x)
            return fy        
        def d2Yd(self):
            fy = dict(self._base.par)
            for k in fy:
                fy[k] = [0. for i in range(self._base.nexp)]
                for l in fy:
                    if l == k:
                        continue
                    fy[k+l] = [0. for i in range(self._base.nexp)]
            for x in self.x:
                for n in self.base.nexp:
                    fy['A'][n] += math.exp(-self._base.par['E'][n]*x)
                    fy['E'][n] += -x*self._base.par['A'][n]*math.exp(-self._base.par['E'][n]*x)
            return fy
            
    
    
class ML_PostAnalyze_PDF:     
    def __init__(self, params): 
        self.headline = 'PDF data analysis with ML:'
        self.tagX = 'Training'
        self.tagY = 'Target'
        self.tagAnly = 'Analysis result:'
        self.keylist = ('ndata.tr', 'ndata.bc', 'ndata.lb', 'ndata.unlbl', 'score', 'p', 'z', 't')
        # Catogerize results arrording to keys: 'p', 'z', 't', etc
        try:
            self.keys = params['keys']
        except:
            # intrinsic keys
            self.keys = ( 'p', 'z', 't')
        self.ml = params['tdset']
        # Plot features as in params['plot.xxx']: {'type', xlabel', 'ylabel', 'err', 'colors', 'shapes', ... }
        self.plot = []
        try:
            self.plot.append(dict[params['plot']])
        except:
            print("Warning: no default feature as 'plot'\n")
        for pk in params.key:
            if re.search('(\(^<=plot)(\.)', pk) is not None: 
                self.plot.append(dict(params[pk]))
        try:
            self.ddir = params['indir']
            self.dfile = []
        except:
            self.ddir = None
            self.dfile = params['infile']
        self.adir = params['outdir']
        if self.ddir is not None:
            d1, d2, df = os.walk(self.ddir)
            for f in df:
                if re.search('^'+self.ml+'.\d+', f) is not None:
                    self.dfile.append(df)
        if isinstance(self.dfile, str):
            self.dfile = [self.dfile]
        self.setup_datatable()
        
        
    def setup_datatable(self):
        self.data_table = pandas.DataFrame()
        data = {}
        data['p'] = {}
        data['z'] = {}
        data['t'] = {}
        data['p']['x'] = []
        data['p']['y'] = []
        data['z']['x'] = []
        data['z']['y'] = []
        data['t']['x'] = []
        data['t']['y'] = []
        data['ndata'] = {}
        data['ndata']['tr'] = []
        data['ndata']['bc'] = []
        data['ndata']['lbl'] = []
        data['ndata']['unlbl'] = []
        data['score'] = {}
        data['score']['R'] = []
        data['score']['I'] = []
        data['meanPrdt'] = {}
        data['meanPrdt']['R'] = []
        data['meanPrdt']['I'] = []
        data['stdPrdt'] = {}
        data['stdPrdt']['R'] = []
        data['stdPrdt']['I'] = []
        data['meanObsv'] = {}
        data['meanObsv']['R'] = []
        data['meanObsv']['I'] = []
        data['stdObsv'] = {}
        data['stdObsv']['R'] = []
        data['stdObsv']['I'] = []
        for file in self.dfile:
            with open(self.ddir+'/'+file, 'r') as pf:
                readpz = False
                readt = False
                readd = False
                dflag = 0
                for line in pf:
                    if readd:
                        if 'score' in line:
                            ss = str(re.search('\[(.*?\])', line).group(0)).split()
                            ss[0].split('[')
                            data['score']['R'].append(ss[0].split('[')[1])
                            data['score']['I'].append(ss[1].split(']')[0])
                        if '+-' in line and dflag > 0:
                            yy = str(re.search('(<=Y\d+ ).*+', line).group(0)).split()
                            try:
                                yid = int(re.search('\d+', line.split()[0]).group(0))
                            except:
                                yid = dflag
                            if yid%2 == 0:
                                data['meanPrdt']['R'].append(yy[0])
                                data['stdPrdt']['R'].append(yy[2])
                                data['meanObsv']['R'].append(yy[4])
                                data['stdObsv']['R'].append(yy[6])
                            else:
                                data['meanPrdt']['I'].append(yy[0])
                                data['stdPrdt']['I'].append(yy[2])
                                data['meanObsv']['I'].append(yy[4])
                                data['stdObsv']['I'].append(yy[6])
                            dflag -= 1
                        if dflag == 0:
                            readd = False
                            readt = True
                    if readt and 'data' in line:
                        if self.tagY in line:
                            ll = list(str(re.search('(<=t\= )[\d]'), line).group(0))
                            dflag *= len(ll)
                            data['t']['y'].extend(ll)
                            del ll
                        elif self.tagX in line:
                            data['t']['x'].extend(list(str(re.search('(<=t\= )\[(\d, *)\d\]'), line).group(0)))
                        readt = False
                    if readpz and 'data' in line:
                        if self.tagY in line:
                            ll = list(re.findall('(<=p\= )((\d+)|(-\d+))', line))
                            dflag *= len(ll)
                            data['p']['y'].extend(ll)
                            del ll
                            data['z']['y'].extend(list(re.findall('(<=z\= )((\d+)|(-\d+)|(None))', line)))
                        elif self.tagX in line:
                            data['p']['x'].extend(list(str(re.search('(<=p\= )\[(((\d+)|(-\d+)), *)((\d+)|(-\d+))\]'), line).group(0)))
                            data['z']['x'].extend(list(str(re.search('(<=z\= )\[((-\d+, )(\d, )(None, )*)((\d+)|(-\d+)|(None))\]'), line).group(0)))
                            readpz = False
                            readt = True
                    if line == self.headline:
                        readpz = True
                        readt = False
                        dflag = 2 # Real & Imag
                    if self.tagAnly in line:
                        readd = True
                        nd = str(re.search('\d/\d/\d', line).group(0)).split('/')
                        data['ndata']['tr'].extend(int(nd[0]))
                        data['ndata']['bc'].extend(int(nd[1]))
                        data['ndata']['unlbl'].extend(int(nd[2]))
                        data['ndata']['lbl'].extend(int(nd[0])+int(nd[1]))
        self.data_table = pds.DataFrame(data=data)
        return
    
    def print_data(self, keys):
        return


                                                

In [8]:
import numpy as np
a = [(0,1), (1,2), (0,1)]
print(np.array(a)[:,0])

[0 1 0]
